# #160 Bearing Schedule
<i>Create bearing schedule as per EN1990:2023</i>
***

In [ ]:
file_path = "DataFiles/160 BearingSchedule.xlsx" # Path to the Excel file containing load case IDS

point_supports = [57, 59]

In [ ]:
reaction_components = ['FX', 'FY', 'FZ']

In [ ]:
# Read definitions file
import numpy as np
import pandas as pd
df = pd.read_excel(file_path, sheet_name="Loadcases", usecols=range(0,3))

for component in reaction_components:
    for opt in ['max', 'min']:
        df[component, opt] = np.nan

df.columns = pd.MultiIndex.from_tuples([(i,'') if np.isscalar(i) else i for i in df.columns])

In [ ]:
import sys; sys.path.append('../') # Reference modules in parent directory
from LPI_23_0 import *
lusas = get_lusas_modeller()

if not lusas.existsDatabase():
    raise Exception("This script will add loadcases to an an existing model, please open a model and run the script again.")

# Reference the current database for convenience
db = lusas.database()

In [ ]:
dict_point_id_to_schedule:dict[int, pd.DataFrame] = dict()
dict_point_id_to_node:dict[int, IFNode] = dict()

for id in point_supports:
    if not db.exists("Point", id) : raise Exception("Point {id} is not available in the open model")
    dict_point_id_to_schedule[id] = df.copy()
    
    point:IFPoint = win32.CastTo(db.getObject("Point", id), "IFPoint")
    dict_point_id_to_node[id] = point.getNodes()[0]

In [ ]:
context:IFResultsContext = lusas.newResultsContext(lusas.view())
obs = context.getCalcResultsSet()
obs.remove("all")
for pid in point_supports:
    obs.add("point", pid)

In [ ]:
def get_results(loadset:IFLoadset, component:str, max_min:str) -> IFResultsComponentSet:

    if(loadset.getTypeCode() == 3):
        env :IFEnvelope = win32.CastTo(loadset, "IFEnvelope")
        if(max_min == "min" and env.isMax()):
            loadset = env.getAssocLoadset()
    elif(loadset.getTypeCode() == 6):
        smart :IFSmartCombination = win32.CastTo(loadset, "IFSmartCombination")
        if(max_min == "min" and smart.isMax()):
            loadset = smart.getAssocLoadset()

    if loadset.needsPrimaryComponent():
        context.setActiveLoadsetAssocVal("Reaction", component, loadset)
    else:
        context.setActiveLoadset(loadset)

    return db.getResultsComponentSet("Reaction", component, "Nodal", context)

In [ ]:
# Loop through each loadcase and get the results
for i, row in df.iterrows():
    id = row['ID','']
    if not isinstance(id, str) and np.isnan(id) : continue    
    id = int(id)
    if not db.existsLoadset(id):
        print (f"Loadcase {id} is not present in the model")
        continue
    
    # Get results
    loadset = db.getLoadset(id)

    for component in reaction_components:
        for max_min in ["max", "min"]:
            results = get_results(loadset, component, max_min)
            iComp = results.getComponentNumber(component)

            for pointID, node in dict_point_id_to_node.items():

                val = results.getContinuousResults(iComp, node, None, None)

                dict_point_id_to_schedule[pointID][component,max_min].iat[i] = val



In [ ]:
with pd.ExcelWriter("Bearing Schedule.xlsx") as writer:
    for pointID, df in dict_point_id_to_schedule.items():
        df.to_excel(writer, sheet_name=f"Point{pointID}", index=False)